# Time Series Exploratory Data Analysis 

### This notebook explores the store data and the German Electric Data 

In [11]:
import pandas as pd
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sma
import seaborn as sns
import prepare
import warnings
warnings.filterwarnings("ignore")
import acquire
from acquire import get_store_item_demand_data

In [12]:
df = acquire.get_store_item_demand_data()

Reading from CSV file. . .
Reading from csv file. . .
Reading from csv file. . .


In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.item_id.unique()

In [ ]:
df.sale_amount.unique()

In [ ]:
df.store_id.unique()

In [ ]:
df.item_brand.unique()

In [ ]:
df.item_name.unique()

In [ ]:
df.item_price.unique()

In [ ]:
df.item_upc12.unique()

In [ ]:
df.item_upc14.unique()

In [ ]:
df.store_address.unique()

In [ ]:
df.store_city.unique()

In [ ]:
df.store_state.unique()

In [ ]:
df.store_zipcode.unique()

In [17]:
df['month'] = df.index.month

df['day_of_week'] = df.index.day_name()
    #create a feature sales total that is the product of quantity and sale price
df['sales_total'] = df.quantity * df.item_price

AttributeError: 'Int64Index' object has no attribute 'month'

In [14]:
df.sale_date = df.sale_date.apply(lambda date: date[:-13])

In [15]:
df.sale_date = pd.to_datetime(df.sale_date, format='%a, %d %b %Y')

In [18]:
df = df.set_index('sale_date').sort_index()

In [19]:
df = df.rename(columns={'sale_amount': 'quantity'})

In [20]:
df['month'] = df.index.month

df['day_of_week'] = df.index.day_name()
    #create a feature sales total that is the product of quantity and sale price
df['sales_total'] = df.quantity * df.item_price

In [21]:
df.to_csv('acquired_and_prepared_store_data.csv')

In [22]:
df.head()

,Unnamed: 0,item_id,quantity,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode,month,day_of_week,sales_total
sale_date,,,,,,,,,,,,,,,,,
2013-01-01,0,1,13.0,1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,1,Tuesday,10.92
2013-01-01,211816,12,26.0,211817,7,Mueller,Mueller Sport Care Basic Support Level Medium ...,8.40,74676640211,74676640211,12018 Perrin Beitel Rd,San Antonio,TX,78217,1,Tuesday,218.40
2013-01-01,832656,46,27.0,832657,7,Mama Marys,Pizza Sauce,4.65,35457770664,35457770664,12018 Perrin Beitel Rd,San Antonio,TX,78217,1,Tuesday,125.55
2013-01-01,213642,12,54.0,213643,8,Mueller,Mueller Sport Care Basic Support Level Medium ...,8.40,74676640211,74676640211,15000 San Pedro Ave,San Antonio,TX,78232,1,Tuesday,453.60
2013-01-01,215468,12,35.0,215469,9,Mueller,Mueller Sport Care Basic Support Level Medium ...,8.40,74676640211,74676640211,735 SW Military Dr,San Antonio,TX,78221,1,Tuesday,294.00


# Split the Data 

In [ ]:
#look at the range of the index 
#four total years of data 
df.index.min(), df.index.max()

In [ ]:
#train test split 
#75% train, 25% test 
#split the data with everything up to 2016 
#test is only 2017 
train = df[:'2016'] 
test = df['2017']

plt.plot(train.index, train.sales_total)
plt.plot(test.index, test.sales_total)

- Key Takeaway: 
    - 3 years of training data, 1 year of test data 
    - test data is most recent 

In [ ]:
train.resample('D').sales_total.mean().plot()
test.resample('D').sales_total.mean().plot()

- Key Takeaways: 
    - The business becomes more profitable every year 
    - The business seems to be most profitable mid year (summer)

In [ ]:
y = train.sales_total 
y.head()

### Visualization Options 

In [ ]:
#distributions of target variable 
train.sales_total.plot.hist(title = 'Frequency of Sales Total')

- Key Takeaway: 
    - Right Skewed Data 

In [ ]:
#sales by day of the week 
train.groupby('day_of_week').sales_total.mean().plot.bar()

- Key Takeaway: 
    - Pretty decent averages of sales 
    - higher sales on Fridays, Saturdays, and Sundays 

In [ ]:
ax = y.groupby(y.index.month).mean().plot.bar(width = 0.9, ec = 'black')
plt.xticks(rotation = 0)
ax.set(title='Average Sales by Month', xlabel='Month', ylabel='Total Sales $')

- Key Takeaway: 
    - More sales in the summer months 
    - The kids are possibly out of school and need to eat at home 
    

In [ ]:
train.day_of_week = train.index.strftime('%a')
train.day_of_week = pd.Categorical(train.day_of_week, categories=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], ordered=True)

In [ ]:
train.head()

In [ ]:
#Sales by day of the week 
sns.boxplot(data = train, y= 'sales_total', x= 'day_of_week')

- Key Takeaways: 
     - More Sales on Friday, Saturday, Sunday
     - More outliers on Thursdays (possibly coupon day?)

# Time Series Visuals 

### Target Over Time 

In [ ]:
y = train.sales_total

In [ ]:
y.plot()

### Target Mean over Periods of Time 

In [ ]:
y.resample('W').mean().plot(title = 'Weekly Average')

In [ ]:
y.resample('M').mean().plot(title = 'Monthly Average')

### Visualize change in Target Over Time 

In [ ]:
#week to week changes 
y.resample('W').mean().diff().plot(title='Weekly Sales Over Time')

In [ ]:
#monthly
y.resample('M').mean().diff().plot(title='Monthly Sales Over Time')

# Seasonal Plot 

In [ ]:
y.groupby([y.index.year, y.index.month]).mean().unstack(0).plot(title='Sales each Year')
plt.xlabel('Month')
plt.ylabel('Total Sales $')

### Seasonal Subseries Plots

In [ ]:
table = y.groupby([y.index.year, y.index.month]).mean().unstack()

fig, axs = plt.subplots(1, 12, sharey=True, sharex=True)
for ax, (month, subset) in zip(axs, table.iteritems()):
    subset.plot(ax=ax, title=month)
    ax.hlines(subset.mean(), *ax.get_xlim())
    ax.set(xlabel='')

fig.suptitle('Seasonal Subseries Plot') # super-title for the overall figure
fig.subplots_adjust(wspace=0)

### LAG Plots 

In [ ]:
plt.scatter(y, y.shift(-1))
plt.xlabel('$y$')
plt.ylabel('$y_{t + 1}$')
plt.title('Lag plot with lag=1')

### Autocorrelation Plot 

In [ ]:
pd.plotting.autocorrelation_plot(train.sales_total.resample('W').mean())


### Germany Dataset 

In [ ]:
df = acquire.german_electric()

In [ ]:
df = prepare.prep_german_electric_data(df)

In [ ]:
df = df.dropna()

In [ ]:
df.head()

In [ ]:
df.shape

### Split the Data 

In [ ]:
#look at the range of index 
df.index.min(), df.index.max()
# 6 years of data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#train test split 
train = df[:'2015'] 
test = df['2016' and '2017']

plt.plot(train.index, train.Consumption)
plt.plot(test.index, test.Consumption)

In [ ]:
train.resample('D').Consumption.mean().plot()
test.resample('D').Consumption.mean().plot()

### Distribution 

In [ ]:
train.Consumption.plot.hist(title = 'Consumption Distribution')

In [ ]:
train.groupby('year').Consumption.mean().plot.bar()

In [ ]:
train.resample('M').mean().plot(title = 'Weekly Consumption')

In [ ]:
train.resample('3M').mean().plot(title = '3 Month Average')

In [ ]:
train.resample('W').mean().diff().plot(title='Average Weekly')

In [ ]:
A = train.Consumption
B = train.Wind
C = train.Solar 
D = train['Wind+Solar']

In [ ]:
A.resample('W').mean().diff().plot(title='Average Weekly Consumption')

In [ ]:
B.resample('W').mean().diff().plot(title='Average Weekly Wind')

In [ ]:
C.resample('W').mean().diff().plot(title='Average Weekly Solar')

In [ ]:
D.resample('W').mean().diff().plot(title='Average Weekly Wind plus Solar')

In [ ]:
A.resample('M').mean().diff().plot(title='Average Monthly Consumption')

In [ ]:
B.resample('M').mean().diff().plot(title='Average Monthly Wind')

In [ ]:
C.resample('M').mean().diff().plot(title='Average Monthly Solar')

In [ ]:
D.resample('M').mean().diff().plot(title='Average Monthly Wind and Solar')

In [ ]:
A.groupby([A.index.year, A.index.month]).mean().unstack(0).plot(title='Seasonal Plot')
plt.xlabel('Month')
plt.ylabel('Total Consumption')